In [1]:
import json
import pandas as pd

anime_themes_json = json.load(open("./raw_data/animethemes_data.json", "r"))
my_anime_list = json.load(open("./raw_data/myanimelist_data.json", "r"))
df_my_anime_list = pd.DataFrame(my_anime_list)


In [2]:
animes_parsed = []
anime_themes_parsed = []
for anime in anime_themes_json:
    anime = anime.copy()
    if "animethemes" in anime.keys():
        for anime_theme in anime["animethemes"]:
            if len(anime_theme["animethemeentries"]) > 0:
                at = {"type": anime_theme["type"]}
                at["anime_id"] = anime["id"]
                if "song" in anime_theme.keys() and anime_theme["song"] is not None:
                    at["song"] = anime_theme["song"]["title"]
                anime_theme_entry = anime_theme["animethemeentries"][0]
                at["spoiler"] = anime_theme_entry["spoiler"]
                at["nsfw"] = anime_theme_entry["nsfw"]

                if (
                    "videos" in anime_theme_entry.keys()
                    and len(anime_theme_entry["videos"]) > 0
                ):
                    video = anime_theme_entry["videos"][0]
                    at["video_link"] = video["link"]
                    at["video_resolution"] = video["resolution"]

                anime_themes_parsed.append(at)

        anime.pop("animethemes")

    if "animesynonyms" in anime.keys():
        anime["synonyms"] = []
        for anime_synonym in anime["animesynonyms"]:
            anime["synonyms"].append(anime_synonym["text"])

        anime.pop("animesynonyms")

    anime["image"] = anime["images"][0]["link"]
    anime.pop("image")

    animes_parsed.append(anime)


In [3]:
anime_df = pd.DataFrame(animes_parsed)

In [4]:
a = []
for i in df_my_anime_list.head(500)["title"].str.lower():
    if i not in anime_df["name"].str.lower().values:
        a.append(i)
        print(i)

print(len(a))

hunter x hunter (2011)
mirai nikki (tv)
ansatsu kyoushitsu
shokugeki no souma
jojo no kimyou na bouken (tv)
psycho-pass
black clover
yahari ore no seishun love comedy wa machigatteiru.
ansatsu kyoushitsu 2nd season
kaguya-sama wa kokurasetai? tensai-tachi no renai zunousen
howl no ugoku shiro
sakura-sou no pet na kanojo
haikyuu!! karasuno koukou vs. shiratorizawa gakuen koukou
fate/stay night: unlimited blade works
tonari no totoro
fairy tail (2014)
5-toubun no hanayome
hellsing ultimate
haikyuu!! to the top
akira
hotarubi no mori e
fruits basket 1st season
toki wo kakeru shoujo
haikyuu!! to the top part 2
jujutsu kaisen 0 movie
yamada-kun to 7-nin no majo
maou gakuin no futekigousha: shijou saikyou no maou no shiso, tensei shite shison-tachi no gakkou e kayou
kono subarashii sekai ni shukufuku wo! movie: kurenai densetsu
shokugeki no souma: san no sara - tootsuki ressha-hen
5-toubun no hanayome ∬
hotaru no haka
psycho-pass 2
suki tte ii na yo.
kenpuu denki berserk
koukaku kidoutai
nag

In [10]:
from fuzzywuzzy import fuzz

anime_theme_df_test = pd.DataFrame(animes_parsed)
my_anime_list_df_test = pd.DataFrame(my_anime_list).head(500)

my_anime_list_df_test["title_parsed"] = my_anime_list_df_test["title"].str.lower()

anime_theme_df_test = anime_theme_df_test.rename(columns={"name": "title"})
anime_theme_df_test["title_parsed"] = anime_theme_df_test["title"].str.lower()
anime_theme_df_test["title_matched"] = anime_theme_df_test["title_parsed"]


def get_matches(x, col):
    ratios = []
    for i in col:
        ratios.append(fuzz.ratio(x, i))

    print(x, col[ratios.index(max(ratios))])

    return col[ratios.index(max(ratios))]


def fuzzy_merge(df1, df2, col):
    df2[col] = df2[col].apply(lambda x: get_matches(x, df1[col]))

    # merge the DataFrames into one
    df3 = df1.merge(df2)

    return df3


# view final DataFrame
df_merged = fuzzy_merge(my_anime_list_df_test, anime_theme_df_test, "title_parsed")


.hack//liminality kimi ni todoke
.hack//roots charlotte
.hack//sign black lagoon
.hack//tasogare no udewa densetsu yakusoku no neverland 2nd season
07-ghost gosick
100-man no inochi no ue ni ore wa tatteiru yahari ore no seishun love comedy wa machigatteiru.
11eyes 91 days
12-sai.: chicchana mune no tokimeki shinmai maou no testament
12-sai.: chicchana mune no tokimeki 2nd season tensei shitara slime datta ken 2nd season
18if relife
2.43: seiin koukou danshi volley-bu danshi koukousei no nichijou
22/7 fate/zero
22/7: 8＋3=? fate/stay night
25-sai no joshikousei kami nomi zo shiru sekai
3 choume no tama: uchi no tama shirimasen ka? shokugeki no souma: shin no sara
3-gatsu no lion 3-gatsu no lion
3-gatsu no lion 2nd season 3-gatsu no lion
30-sai no hoken taiiku grisaia no kajitsu
3d kanojo: real girl aho girl
3d kanojo: real girl 2nd season tokyo ghoul:re 2nd season
7 seeds 91 days
7 seeds 2nd season fate/zero 2nd season
91 days 91 days
a-channel clannad
a.i.c.o.: incarnation boruto: naru

In [8]:
df_merged[df_merged["title_matched"] != df_merged["title_parsed"]]

,rank,title,popularity_score,quality_score,title_parsed,id,slug,year,season,synopsis,created_at,updated_at,deleted_at,images,synonyms,title_matched


In [7]:
df_merged[df_merged["title_parsed"] == "haikyuu!! to the top"]


,rank,title,popularity_score,quality_score,title_parsed,id,slug,year,season,synopsis,created_at,updated_at,deleted_at,images,synonyms,title_matched
